In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output, State
import base64
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


# CRUD Python module file name and class name
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################

# Connect to database via CRUD Module
username = "aacuser"
password = "password123"
HOST = 'localhost' 
PORT = 27017 
DB = 'aac' 
COL = 'animals' 
db = AnimalShelter(username, password, HOST, PORT, DB, COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())


app.layout = html.Div([

    # Grazioso Salvare’s logo placement
    html.A(html.Img(src='data:image/png;base64,{}'.format(encoded_image.decode()), style={'width': '64px'}), href='http://www.snhu.edu/', target='_blank'),
    
    # unique identifier
    html.Header("Programmed by: Crailin", style={
                'position': 'absolute',
                'top': '10px',
                'right': '10px',
                'color': 'green',
                'fontSize': '24px'}),
    
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    
    # user interface components
    html.Div([
    html.B('Rescue Type:'),
    dcc.Checklist(
                id='rescue_type',
                options=[
                    {'label': 'Water ', 'value': '1'},
                    {'label': 'Mountain or Wilderness ', 'value': '2'},
                    {'label': 'Disaster or Individual Tracking ', 'value': '3'}
                ],
                value=[], # default
                inline=True,
                labelStyle={'margin': '4px'}
                ),
    html.Button('Reset', id='btn_reset', n_clicks=0, style={'display': 'inline-block', 'margin-left': '12px'}),
    ], style={'display': 'flex', 'align-items': 'center'}),
    html.Hr(),
    
    # setup data table and options
    dash_table.DataTable(id='datatable-id',
                         columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
                         data=df.to_dict('records'),
                        row_selectable = "single",
                        sort_action="native",
                        sort_mode="multi",
                        page_action="native",
                        page_size= 200,
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################

btn_reset_n_clicks = 0
@app.callback(Output('datatable-id','data'),
              Output('rescue_type', 'value'),
              [Input('rescue_type', 'value'),
               Input('btn_reset', 'n_clicks')])
def update_dashboard(rescue_type, reset):
    global btn_reset_n_clicks
    data = df.to_dict('records')
    
    # reset all records 
    if reset > btn_reset_n_clicks:
        btn_reset_n_clicks = reset
        return data, []
    
    # check filter options
    queries = []
    for value in rescue_type:
        
        if value == '1':
            breed_query1 = {"$or": [
            {"breed" : "Chesapeake Bay Retriever"},
            {"breed" : "Labrador Retriever Mix"},
            {"breed" : "Newfoundland"}]}
            sex_query1 = {"sex_upon_outcome" : "Intact Female"}
            age_query1 = {"age_upon_outcome_in_weeks" : {"$gte": 26, "$lte": 156}}
            query1 = {'$and': [breed_query1, sex_query1, age_query1]}
            queries.append(query1)
            
        if value == '2':
            breed_query2 = {"$or": [
            {"breed" : "German Shepherd"},
            {"breed" : "Alaskan Malamute"},
            {"breed" : "Old English Sheepdog"},
            {"breed" : "Siberian Husky"},
            {"breed" : "Rottweiler"}]}
            sex_query2 = {"sex_upon_outcome" : "Intact Male"}
            age_query2 = {"age_upon_outcome_in_weeks" : {"$gte": 26, "$lte": 156}}
            query2 = {'$and': [breed_query2, sex_query2, age_query2]}
            queries.append(query2)
            
        if value == '3':
            breed_query3 = {"$or": [
            {"breed" : "Doberman Pinscher"},
            {"breed" : "German Shepherd"},
            {"breed" : "Golden Retriever"},
            {"breed" : "Bloodhound"},
            {"breed" : "Rottweiler"}]}
            sex_query3 = {"sex_upon_outcome" : "Intact Male"}
            age_query3 = {"age_upon_outcome_in_weeks" : {"$gte": 20, "$lte": 300}}
            query3 = {'$and': [breed_query3, sex_query3, age_query3]}
            queries.append(query3)
    
    # combine selected queries
    if queries:
        if len(queries) == 1:
            result = list(db.read(queries[0]))
        elif len(queries) > 1:
            result = list(db.read({'$and': queries}))

        # format pymongo result
        for doc in result:
            doc.pop('_id', None)  # removes '_id'

        return result, rescue_type
    
    # no change to dashboard
    return dash.no_update


# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    
    if not viewData:
        return dash.no_update
    
    # filter for specific breeds
    dogBreeds = ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland", "German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky", "Rottweiler", "Doberman Pinscher", "Golden Retriever", "Bloodhound"]
    dff = pd.DataFrame.from_dict(viewData)
    dogs_df = dff[dff['breed'].isin(dogBreeds)]

    return [dcc.Graph(figure = px.pie(dogs_df, names='breed', title='Preferred Animals'))]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):
    
    # return empty map if nothing is selected
    if not index:
        return [dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[])]
    
    row = index[0] # row index
    dff = pd.DataFrame.from_dict(viewData) # data for marker
    
    # return map with animal location marker
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'},
           center=[dff.iloc[row,13],dff.iloc[row,14]], zoom=10, children=[
           dl.TileLayer(id="base-layer-id"),
           # Marker with tool tip and popup
           # Column 13 and 14 define the grid-coordinates for 
           # the map
           # Column 4 defines the breed for the animal
           # Column 9 defines the name of the animal
           dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]],
              children=[
              dl.Tooltip(dff.iloc[row,4]),
              dl.Popup([
                 html.H1("Animal Name"),
                html.P(dff.iloc[row,9])
             ])
          ])
       ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

INITIALIZING CONNECTION
{'ok': 1.0,
 'users': [{'_id': 'admin.aacuser',
            'db': 'admin',
            'mechanisms': ['SCRAM-SHA-1', 'SCRAM-SHA-256'],
            'roles': [{'db': 'aac', 'role': 'readWrite'}],
            'user': 'aacuser',
            'userId': Binary(b'\xf1\xfb\xaf\xde|\xdfMV\xbc\x1a\xd0lFd\x13;', 4)}]}

Dash app running on https://greenjaguar-nextimitate-3000.codio.io/proxy/8050/
